**Zadanie 1. (8p)** Napisz program MCR (Mistrz Ciętej Riposty), który umożliwia przeprowadzenia
dialogu człowieka z komputerem, w którym wypowiedzi komputera są

a) wzięte z wikicytatów,

b) pasujące w pewnym stopniu do wypowiedzi przedmówcy.

Należy zaproponować podział dłuższych cytatów na „minicytaty”, które będą potencjalnymi
odpowiedziami. Program dodatkowo powinien:

1. tworzyć sobie wewnętrzny ranking pasujących wypowiedzi do danego pytania,
2. losować odpowiedź w ten sposób, by odpowiedzi na szczycie rankingu miały dużo większą
szansę być wylosowanymi (losowanie powinno być możliwe do wyłączenia, w tej wersji odpowiedź jest po prostu liderem rankingu)
3. korzystać z prostego stemmera lub lematyzacji,
4. (generalnie) nie powtarzać wypowiedzi w ramach jednego dialogu,
5. pilnować, by wypowiedź nie była nigdy kopią (lub prawie kopią) wypowiedzi poprzedzającej
6. reagować jakimś ogólnym cytatem w przypadku nieznalezienia niczego pasującego
7. unikać takich słów jak on, go, ich, których znaczenie zależy od kontekstu.

Przykładowe dialogi hipotetycznego MCR-a znajdują się na SKOS-ie. Do tworzenia Mistrza
Ciętej Riposty jeszcze wrócimy na kolejnych listach.

In [1]:
import csv
import random
from collections import defaultdict

In [2]:
BASE_FORMS_FILE_PATH = 'data/polimorfologik-2.1.txt'
TOKENIZED_QUOTES_FILE_PATH = 'data/tokenized_quotes.txt'

# Search engine

Taken mostly verbatim from _remote-lab1_

In [3]:
BASE_FORMS = {}

with open(BASE_FORMS_FILE_PATH) as f:
    for base_form, word, *_ in csv.reader(f, delimiter=';'):
        BASE_FORMS[word] = base_form

In [24]:
def base(words):
    result = set()
    for word in words:
        if base_form := BASE_FORMS.get(word):
            result.add(base_form)
    return result

In [25]:
QUOTES = []
INDEX = defaultdict(set)

with open(TOKENIZED_QUOTES_FILE_PATH) as f:
    i = 0
    for line in f:
        quotes = line.replace('?', '.').replace('!', '.').split('.')
        for quote in quotes:
            words = quote.split()
            if not words:
                continue
            QUOTES.append(words)
            for base_ in base(words):
                INDEX[base_].add(i)
            i += 1

In [26]:
def find_quotes(query):
    # Not intersection here, because it's for a chat bot
    # I presume it could find nothing for most inputs
    quote_ids = set.union(*(INDEX[qword] for qword in base(query)))
    return [QUOTES[id_] for id_ in quote_ids]

# Chat bot

In [29]:
RANDOM = False
USED_RESPONSES = []
FORBIDDEN_WORDS = {'go', 'on', 'ich'}

EXACT_MATCH_MODIFIER = 5
BASE_FORM_MATCH_MODIFIER = 5
FORBIDDEN_WORD_MODIFIER = -10

def score(resp, user_input):
    if resp in USED_RESPONSES:
        return -1000
    if resp == user_input:
        return -1000
    exact_matches = len([
        uword for uword in user_input
        if uword in resp
    ])
    base_form_matches = len([
        buword for buword in base(user_input)
        if buword in base(resp)
    ])
    forbidden_words = len([
        rword for rword in resp
        if rword in FORBIDDEN_WORDS
    ])
    return (
        exact_matches * EXACT_MATCH_MODIFIER
        + base_form_matches * BASE_FORM_MATCH_MODIFIER
        + forbidden_words * FORBIDDEN_WORD_MODIFIER
    )

def rank_responses(responses, user_input):
    return [(resp, score(resp, user_input)) for resp in responses]

def choose_resp(ranked_responses):
    if RANDOM:
        resps, scores = zip(*ranked_responses_responses)
        resp = random.choices(resps, weights=scores)[0]
    else:
        resp = max(ranked_responses, key=lambda rs: rs[1])[0]
    USED_RESPONSES.append(resp)
    return resp

In [33]:
while True:
    user_input = input('> ').split()
    quotes = find_quotes(user_input)
    resps = rank_responses(quotes, user_input)
    resp = choose_resp(resps)
    print(' '.join(resp), end='\n\n')

> co możesz powiedzieć o teorii kategorii
i sam się wciągnąłem… nie lubię o tym mówić , bo co mogę powiedzieć

> naprawdę lubisz takie abstrakcyjne rzeczy
straszne rzeczy tak naprawdę nie są takie jak na filmie

> jakie lubisz filmy z takich które oglądałem
życie to jest co innego : twoi rodzice , dom , nachylające się nad tobą twarze , dzieciństwo i młodość , przyjaciele i znajomi , ludzie i uczucia między ludźmi , twoje życie , jaki zawód wybierzesz , jakie książki , filmy , potrawy lubisz , czy spoglądasz nocą w niebo , czy zasypiasz na łące pod kwiatami , czy odurzył cię zapach lasu , co zapamiętasz , twoje życie , kilka osób , kilka przedmiotów , kilka miejsc , żona , dzieci , kilka osób , które kochasz i które ciebie kochają , i żebyś mógł zapewnić im szczęście , widział , jak dorastają , starzeją się , dał , co najlepsze , wziął , co najlepsze , żeby wnuki twoją twarz nad sobą widziały , i żeby tak się domknęło , kilka osób , kilka przedmiotów , kilka miejsc , twoje życie

> to 

KeyboardInterrupt: 